In [8]:
from random import choices
import cirq


### Initiate Constants

In [17]:
num_bits = 10

encode_gates = {0: cirq.I, 1: cirq.X} # Gates that produce the 0 and 1 quantums states, which represents the keys
basis_gates = {"Z": cirq.I, "X": cirq.H} # Gates that will create the correct qubit encryption

### Create Alice Key and Alice Encryption Bases

In [13]:
alice_key = choices([0, 1], k = num_bits)
alice_key

[0, 0, 1, 0, 1, 1, 0, 1, 1, 0]

In [16]:
alice_bases = choices(['Z', 'X'], k = num_bits)
alice_bases

['X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z']

### Encode Alice Key into Qubits

In [20]:
alice_circuit = cirq.Circuit()
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

for bit in range(num_bits):

 encode_value = alice_key[bit]
 encode_gate = encode_gates[encode_value]

 basis_value = alice_bases[bit]
 basis_gate = basis_gates[basis_value]

 qubit = qubits[bit]
 alice_circuit.append(encode_gate(qubit))
 alice_circuit.append(basis_gate(qubit))

alice_circuit

q0: ───I───H───

q1: ───I───I───

q2: ───X───H───

q3: ───I───I───

q4: ───X───I───

q5: ───X───I───

q6: ───I───I───

q7: ───X───I───

q8: ───X───I───

q9: ───I───I───

### Alice sends her encrypted key to Bob

> _Vroom Vroom Vroom!_

### Bob picks decryption bases

In [21]:
bob_bases = choices(['Z', 'X'], k = num_bits)
bob_bases

['X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z']

### Create Bob's decryption circuit

In [24]:
bob_circuit = cirq.Circuit()
for bit in range(num_bits):
    basis_value = bob_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    bob_circuit.append(basis_gate(qubit))
    
bob_circuit.append(cirq.measure(qubits, key = 'bob decryption'))
bob_circuit

q0: ───H───M('bob key')───
           │
q1: ───H───M──────────────
           │
q2: ───I───M──────────────
           │
q3: ───I───M──────────────
           │
q4: ───H───M──────────────
           │
q5: ───I───M──────────────
           │
q6: ───I───M──────────────
           │
q7: ───I───M──────────────
           │
q8: ───H───M──────────────
           │
q9: ───I───M──────────────

### Create Bob key

In [26]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]
bob_key

array([0, 0, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int8)

### Create final shared key

In [27]:
final_alice_key = []
final_bob_key = []

for bit in range(num_bits):
    if alice_bases[bit] == bob_bases[bit]:
        final_alice_key.append(alice_key[bit])
        final_bob_key.append(bob_key[bit])
final_alice_key


[0, 0, 1, 0, 1, 0]